In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [102]:
# Load the dataset
data = pd.read_csv(r"C:\Users\leere\OneDrive\Desktop\RAW DATA\CGMBET_ALL_21_25.CSV")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 12743: invalid start byte

In [95]:
# Select relevant features for prediction
features = [
    "Rolling avg ball possession (Total)",
    "Rolling avg ball possession conceded (Total)",
    "Rolling avg points won (Total)",
    "elo difference",
    "Rolling avg corners (Total)",
    "Rolling avg shots (Total)",
    "Rolling avg shots on target (Total)",
    "Rolling avg corners conceded (Total)",
    "Team elo",
    "Opponent elo"
]

# Target variable
target = "Total match corners"

# Keep only the features and the target column
data = data[features + ["Round", "Team", target]]



In [96]:
# Shift only the rolling averages columns to ensure proper training (e.g., for each team)
rolling_avg_features = [
    "Rolling avg ball possession (Total)",
    "Rolling avg ball possession conceded (Total)",
    "Rolling avg points won (Total)",
    "Rolling avg corners (Total)",
    "Rolling avg shots (Total)",
    "Rolling avg shots on target (Total)",
    "Rolling avg corners conceded (Total)"
]
data[rolling_avg_features] = data.groupby("Team")[rolling_avg_features].shift(1)
# Drop rows with missing values caused by shifting
#data = data[data["Round"]>5]
data = data.dropna()

data

,Rolling avg ball possession (Total),Rolling avg ball possession conceded (Total),Rolling avg points won (Total),elo difference,Rolling avg corners (Total),Rolling avg shots (Total),Rolling avg shots on target (Total),Rolling avg corners conceded (Total),Team elo,Opponent elo,Round,Team,Total match corners
1,53.000000,47.000000,1.000000,-153.0,8.000000,15.000000,3.0,9.0,1797.0,1950.0,2,1. FSV Mainz 05,13
2,45.500000,54.500000,1.000000,30.0,5.000000,12.000000,5.0,10.0,1809.0,1779.0,3,1. FSV Mainz 05,14
3,48.666667,51.333333,0.666667,190.0,6.666667,11.666667,5.0,8.0,1781.0,1591.0,4,1. FSV Mainz 05,15
4,45.000000,55.000000,1.333333,51.0,5.250000,10.000000,4.5,9.5,1795.0,1744.0,5,1. FSV Mainz 05,13
5,46.400000,53.600000,1.000000,53.0,5.600000,9.400000,4.0,8.8,1752.0,1699.0,6,1. FSV Mainz 05,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13069,50.400000,49.600000,1.333333,61.0,3.400000,10.400000,3.6,7.2,1739.0,1678.0,16,Zwolle,14
13070,54.200000,45.800000,1.333333,128.0,4.200000,13.200000,3.4,6.4,1710.0,1582.0,17,Zwolle,10
13071,54.000000,46.000000,0.666667,-38.0,3.800000,12.400000,3.4,6.4,1708.0,1746.0,18,Zwolle,10
13072,57.800000,42.200000,0.333333,-540.0,5.000000,11.400000,2.8,5.8,1684.0,2224.0,19,Zwolle,7


In [97]:
team_filter = data[data["Team"]=="Chelsea"]
team_filter

,Rolling avg ball possession (Total),Rolling avg ball possession conceded (Total),Rolling avg points won (Total),elo difference,Rolling avg corners (Total),Rolling avg shots (Total),Rolling avg shots on target (Total),Rolling avg corners conceded (Total),Team elo,Opponent elo,Round,Team,Total match corners
2767,48.000000,52.000000,0.000000,220.0,4.000000,10.000000,3.00,3.0,1834.0,1614.0,2,Chelsea,10
2768,55.000000,45.000000,1.500000,124.0,4.500000,12.000000,5.50,4.0,1860.0,1736.0,3,Chelsea,8
2769,57.666667,42.333333,1.333333,137.0,4.333333,12.333333,6.00,4.0,1848.0,1711.0,4,Chelsea,9
2770,60.000000,40.000000,2.333333,172.0,4.000000,11.750000,5.25,4.5,1865.0,1693.0,5,Chelsea,11
2771,57.600000,42.400000,2.333333,176.0,4.200000,11.800000,5.20,4.8,1892.0,1716.0,6,Chelsea,10
2772,56.200000,43.800000,3.000000,245.0,5.000000,12.800000,6.00,4.6,1902.0,1657.0,7,Chelsea,14
2773,57.000000,43.000000,2.333333,-85.0,6.200000,14.400000,6.00,4.2,1886.0,1971.0,8,Chelsea,6
2774,55.800000,44.200000,1.333333,93.0,6.400000,14.200000,5.00,3.6,1875.0,1782.0,9,Chelsea,11
2775,52.600000,47.400000,1.333333,158.0,7.200000,15.600000,5.80,3.2,1884.0,1726.0,10,Chelsea,12
2776,53.800000,46.200000,1.333333,-91.0,7.800000,15.600000,5.40,2.8,1880.0,1971.0,11,Chelsea,7


In [98]:
# Define features (X) and target (y)
X = data[features]
y = data[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning with cross-validation
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


KeyboardInterrupt: 

In [77]:
# Evaluate the model based on "over corner markets"
over_corner_wins = (y_pred < y_test).sum()
total_matches = len(y_test)
win_rate = over_corner_wins / total_matches

# Print evaluation results
print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print(f"Over Corner Market Win Rate: {win_rate:.2%}")

Mean Absolute Error: 2.5528818327974276
Over Corner Market Win Rate: 42.16%


In [79]:

# Glue predictions back to the original dataset
data["Predicted Corners"] = np.nan
data.loc[X_test.index, "Predicted Corners"] = y_pred
data

,Rolling avg ball possession (Total),Rolling avg ball possession conceded (Total),Rolling avg points won (Total),elo difference,Rolling avg corners (Total),Rolling avg shots (Total),Rolling avg shots on target (Total),Rolling avg corners conceded (Total),Team elo,Opponent elo,Round,Team,Total match corners,Predicted Corners
1,53.000000,47.000000,1.000000,-153.0,8.000000,15.000000,3.0,9.0,1797.0,1950.0,2,1. FSV Mainz 05,13,9.79
2,45.500000,54.500000,1.000000,30.0,5.000000,12.000000,5.0,10.0,1809.0,1779.0,3,1. FSV Mainz 05,14,NaN
3,48.666667,51.333333,0.666667,190.0,6.666667,11.666667,5.0,8.0,1781.0,1591.0,4,1. FSV Mainz 05,15,NaN
4,45.000000,55.000000,1.333333,51.0,5.250000,10.000000,4.5,9.5,1795.0,1744.0,5,1. FSV Mainz 05,13,11.13
5,46.400000,53.600000,1.000000,53.0,5.600000,9.400000,4.0,8.8,1752.0,1699.0,6,1. FSV Mainz 05,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13069,50.400000,49.600000,1.333333,61.0,3.400000,10.400000,3.6,7.2,1739.0,1678.0,16,Zwolle,14,9.71
13070,54.200000,45.800000,1.333333,128.0,4.200000,13.200000,3.4,6.4,1710.0,1582.0,17,Zwolle,10,NaN
13071,54.000000,46.000000,0.666667,-38.0,3.800000,12.400000,3.4,6.4,1708.0,1746.0,18,Zwolle,10,NaN
13072,57.800000,42.200000,0.333333,-540.0,5.000000,11.400000,2.8,5.8,1684.0,2224.0,19,Zwolle,7,9.86
